In [1]:
import os
import warnings
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC

# titanic data

Najpierw zaloguj się do https://www.kaggle.com/ i przejdź do wyzwania https://www.kaggle.com/c/titanic, aby pobrać 
 * train.csv i test.csv. 

Zapisz je w katalogu data/titanic.

In [2]:
TITANIC_PATH = os.path.join("..", "data", "titanic")

In [3]:
def load_titanic_data(filename, titanic_path=TITANIC_PATH):
    csv_path = os.path.join(titanic_path, filename)
    return pd.read_csv(csv_path)

In [4]:
train_data = load_titanic_data("train.csv")
test_data = load_titanic_data("test.csv")

* Dane są już podzielone na zestaw treningowy i zestaw testów. 
* Jednak dane testowe nie zawierają etykiet: Twoim celem jest wyszkolenie najlepszego modelu, który możesz wykorzystać w danych treningowych, następnie dokonanie swoich przewidywań na danych testowych i przesłanie ich do Kaggle, aby zobaczyć ostateczny wynik.

Rzućmy okiem na kilka pierwszych rzędów zestawu treningowego:

In [5]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


The attributes have the following meaning:

* Survived: that's the target, 0 means the passenger did not survive, while 1 means he/she survived.
* Pclass: passenger class.
* Name, Sex, Age: self-explanatory
* SibSp: how many siblings & spouses of the passenger aboard the Titanic.
* Parch: how many children & parents of the passenger aboard the Titanic.
* Ticket: ticket id
* Fare: price paid (in pounds)
* Cabin: passenger's cabin number
* Embarked: where the passenger embarked the Titanic
* Let's get more info to see how much data is missing:

In [6]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Atrybuty **Age**, **Cabin** oraz **Embarked** są czasami zerowe (mniej niż 891 wartości bez wartości null), szczególnie w przypadku **Cabin** (77% ma wartość zerową). Zignorujemy teraz **Cabin** i skupimy się na reszcie. Atrybut **Age** ma około 19% wartości pustych, więc będziemy musieli zdecydować, co z nimi zrobić. Zastąpienie wartości null medianą wieku wydaje się uzasadnione.

Atrybuty **Name** i **Ticket** mogą mieć pewną wartość, ale będą one nieco trudne do przekształcenia w użyteczne liczby. Na razie będziemy je ignorować.

Rzućmy okiem na atrybuty liczbowe:

In [7]:
train_data = train_data.drop(["Ticket", "Cabin"], axis=1)  
test_data = test_data.drop(["Ticket", "Cabin"], axis=1)

Zobaczmy że w kolumnie **Name** mamy informacje o pozycji/tytule pasażera. Możemy to wykorzystać:

In [8]:
def extract_unique_titles(df):
    titles = train_data['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    return titles.unique()

unique_titles = extract_unique_titles(train_data)
print(unique_titles)

['Mr' 'Mrs' 'Miss' 'Master' 'Don' 'Rev' 'Dr' 'Mme' 'Ms' 'Major' 'Lady'
 'Sir' 'Mlle' 'Col' 'Capt' 'Countess' 'Jonkheer']


<>:2: SyntaxWarning: invalid escape sequence '\.'
<>:2: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipykernel_61308/3695386273.py:2: SyntaxWarning: invalid escape sequence '\.'
  titles = train_data['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)


Dodajmy do naszych danych kolejną kolumnę z tytułami:

In [9]:
def add_title_column(df, unique_titles):
    title_to_index = {title: index for index, title in enumerate(unique_titles)}
    df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False).map(title_to_index)
    return df

# Użycie funkcji
train_data = add_title_column(train_data, unique_titles)

<>:3: SyntaxWarning: invalid escape sequence '\.'
<>:3: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipykernel_61308/389706244.py:3: SyntaxWarning: invalid escape sequence '\.'
  df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False).map(title_to_index)


In [10]:
train_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Fare             0
Embarked         2
Title            0
dtype: int64

In [11]:
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Title
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208,0.936027
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429,1.725341
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400,0.000000
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200,0.000000
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000,2.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200,16.000000


* Tylko 38% przeżyło: to wystarczająco blisko do 40%, więc **accuracy** będzie rozsądną miarą do oceny naszego modelu.

Sprawdźmy, czy etykiety przyjmują wartości 0 lub 1:

In [12]:
(train_data["Survived"].isin([0, 1])).all()

True

Nie zapomnij o etykietach:

In [13]:
y = train_data["Survived"]
X = train_data.drop("Survived", axis=1)

Teraz rzućmy okiem na wszystkie atrybuty kategoryczne:

In [14]:
X["Sex"].value_counts()

Sex
male      577
female    314
Name: count, dtype: int64

In [15]:
X["Embarked"].value_counts()

Embarked
S    644
C    168
Q     77
Name: count, dtype: int64

In [16]:
# X["Cabin"].value_counts()

Atrybut **Embarked** mówi nam, gdzie pasażer wsiadł: C=Cherbourg, Q=Queenstown, S=Southampton.

Podzielmy zbiór na test/train:

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

Teraz zbudujmy nasze **pipeline** preprocessingu. 

Wykorzystamy DataframeSelector aby wybrać określone atrybuty z DataFrame:

In [18]:
# A class to select numerical or categorical columns 
# since Scikit-Learn doesn't handle DataFrames yet
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.attribute_names]

Zbudujmy **pipeline** dla atrybutów numerycznych:

In [19]:
imputer = SimpleImputer(strategy="median")

num_pipeline = Pipeline([
    ("select_numeric", DataFrameSelector(["Age", "Title", "Pclass", "SibSp", "Parch", "Fare"])),
    ("imputer", SimpleImputer(strategy="median")),
])

In [20]:
num_pipeline.fit_transform(X_train)

array([[ 45.5   ,   0.    ,   1.    ,   0.    ,   0.    ,  28.5   ],
       [ 23.    ,   0.    ,   2.    ,   0.    ,   0.    ,  13.    ],
       [ 32.    ,   0.    ,   3.    ,   0.    ,   0.    ,   7.925 ],
       ...,
       [ 41.    ,   0.    ,   3.    ,   2.    ,   0.    ,  14.1083],
       [ 14.    ,   2.    ,   1.    ,   1.    ,   2.    , 120.    ],
       [ 21.    ,   0.    ,   1.    ,   0.    ,   1.    ,  77.2875]])

Będziemy także potrzebować imputera do kategorycznych kolumn  napisowych (zwykły Imputer nie działa na tych kolumnach):

In [21]:
# Inspired from stackoverflow.com/questions/25239958
class MostFrequentImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.most_frequent_ = pd.Series([X[c].value_counts().index[0] for c in X],
                                        index=X.columns)
        return self

    def transform(self, X, y=None):
        return X.fillna(self.most_frequent_)

Teraz możemy zbudować **pipeline** dla atrybutów kategorycznych:

In [22]:
cat_pipeline = Pipeline([
    ("select_cat", DataFrameSelector(["Sex", "Embarked"])),
    ("imputer", MostFrequentImputer()),
    ("cat_encoder", OneHotEncoder(handle_unknown='ignore')),
])

In [23]:
cat_pipeline.fit_transform(X_train)

<712x5 sparse matrix of type '<class 'numpy.float64'>'
	with 1424 stored elements in Compressed Sparse Row format>

Na koniec połączmy powyższe podejścia:

In [24]:
preprocess_pipeline = FeatureUnion(transformer_list=[
    ("num_pipeline", num_pipeline),
    ("cat_pipeline", cat_pipeline),
])

Teraz mamy fajny **pipeline** przetwarzania wstępnego, który pobiera dane wejściowe i zwraca dane wyjściowe złorzone z liczb, które możemy podać do dowolnego modelu uczenia maszynowego.

# Zad

Robimy StratifiedKFold i znajdujemy optymalne parametry dla

* SVM z jądrem rbf
* SVM z jądrem poly
* SVM liniowego
* Regresji logistycznej

In [25]:
seed = 123
kfold = StratifiedKFold(n_splits=5, random_state=seed, shuffle=True)

In [26]:
def grid_fit(pipe, param_grid):
    grid = GridSearchCV(pipe, param_grid, cv=kfold, return_train_score=True)
    grid.fit(X_train, y_train)

    results = pd.DataFrame(grid.cv_results_)
    scores = np.array(results.mean_test_score)

    return scores, grid.best_estimator_, grid.best_params_

In [27]:
def score_model(model):
    y_pred = model.predict(X_test)

    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    f1 = metrics.f1_score(y_test, y_pred)
    accuracy = metrics.accuracy_score(y_test, y_pred)

    return precision, recall, f1, accuracy

In [28]:
param_grid = {'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100]}

methods = [
    # takes too long
    # ("SVM Linear", Pipeline([('preprocessing', preprocess_pipeline), ('classifier', SVC(kernel="linear"))]),
    #  param_grid),
    ("SVM RBF", Pipeline([('preprocessing', preprocess_pipeline), ('classifier', SVC(kernel="rbf"))])
     , param_grid),
    ("SVM Poly", Pipeline([('preprocessing', preprocess_pipeline), ('classifier', SVC(kernel="poly"))])
     , param_grid),
    ("Logistic Regression", Pipeline([('preprocessing', preprocess_pipeline), ('classifier', LogisticRegression())])
     , param_grid)
]


In [32]:
df = pd.DataFrame(columns=["name", "C", "gamma", "precision", "recall", "f1", "accuracy"])

best_f1 = None
best_model = None

for i, (method, pipe, param_grid) in enumerate(methods):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        scores, best_model, best_params = grid_fit(pipe, param_grid)
        precision, recall, f1, accuracy = score_model(best_model)
        if best_f1 is None or f1 > best_f1:
            best_f1 = f1
            best_model = best_model
        df.loc[i] = [method, best_params['classifier__C'], best_params.get("classifier__gamma", None), *score_model(best_model)]

df

,name,C,gamma,precision,recall,f1,accuracy
0,SVM RBF,100.0,None,0.743243,0.743243,0.743243,0.787709
1,SVM Poly,100.0,None,0.846154,0.297297,0.440000,0.687151
2,Logistic Regression,0.1,None,0.768116,0.716216,0.741259,0.793296


Zajmijmy się danymi testowymi:

In [30]:
test_data = add_title_column(test_data, unique_titles)

In [31]:
predictions = best_model.predict(test_data)
submission_file = "submission.csv"
submission_df = pd.DataFrame({
    "PassengerId": test_data["PassengerId"],
    "Survived": predictions
})
submission_df.to_csv(submission_file, index=False)

![Wynik](scr.png)